In [83]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

In [107]:
import numpy as np
df1 = pd.read_csv(r"C:\Users\drzit\Desktop\challenge qrt 2025\X_train_9po2I7U\X_train\clinical_train.csv")
df2 = pd.read_csv(r"C:\Users\drzit\Desktop\challenge qrt 2025\X_train_9po2I7U\X_train\molecular_train.csv")
target_df = pd.read_csv(r"C:\Users\drzit\Desktop\challenge qrt 2025\target_train.csv")

In [108]:
dg1 = pd.read_csv(r"C:\Users\drzit\Desktop\challenge qrt 2025\X_test_xzVefmA\X_test\clinical_test.csv")
dg2 = pd.read_csv(r"C:\Users\drzit\Desktop\challenge qrt 2025\X_test_xzVefmA\X_test\molecular_test.csv")

In [109]:
df1.info()
df2.info()
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3323 entries, 0 to 3322
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            3323 non-null   object 
 1   CENTER        3323 non-null   object 
 2   BM_BLAST      3214 non-null   float64
 3   WBC           3051 non-null   float64
 4   ANC           3130 non-null   float64
 5   MONOCYTES     2722 non-null   float64
 6   HB            3213 non-null   float64
 7   PLT           3199 non-null   float64
 8   CYTOGENETICS  2936 non-null   object 
dtypes: float64(6), object(3)
memory usage: 233.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10935 entries, 0 to 10934
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              10935 non-null  object 
 1   CHR             10821 non-null  object 
 2   START           10821 non-null  float64
 3   END             10821 non-nu

In [110]:
# Ajoute le nombre de mutation observé chez chaque patient
df1['MUTATION'] = df1['ID'].map(df2['ID'].value_counts()).fillna(0).astype(int)
dg1['MUTATION'] = dg1['ID'].map(dg2['ID'].value_counts()).fillna(0).astype(int)
# Calculer la somme des VAF par ID dans df2
vaf_sum = df2.groupby("ID")["VAF"].max()
vaf_sum2 = dg2.groupby("ID")["VAF"].max()
# Ajouter à df1 une nouvelle colonne VAF_TOT
df1["VAF_TOT"] = df1["ID"].map(vaf_sum).fillna(0)
dg1["VAF_TOT"] = dg1["ID"].map(vaf_sum2).fillna(0)

def is_male(cytogenetic_code):
    if pd.isna(cytogenetic_code):
        return np.nan
    code = str(cytogenetic_code).lower().replace(" ", "")
    if "xy" in code or "x,-y" in code:
        return True
    else:
        return False

df1["SEX_MALE"] = df1["CYTOGENETICS"].apply(is_male).astype("boolean").fillna(False)
dg1["SEX_MALE"] = dg1["CYTOGENETICS"].apply(is_male).astype("boolean").fillna(False)


C:\Users\drzit\AppData\Local\Temp\ipykernel_17800\3932594798.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1["SEX_MALE"] = df1["CYTOGENETICS"].apply(is_male).fillna(False).astype("boolean")
C:\Users\drzit\AppData\Local\Temp\ipykernel_17800\3932594798.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dg1["SEX_MALE"] = dg1["CYTOGENETICS"].apply(is_male).fillna(False).astype("boolean")


In [111]:
vaf_max = df1["VAF_TOT"].max()
print(vaf_max)

0.999


In [112]:
# Pivot binaire avec counts
effect_pivot = pd.crosstab(df2["ID"], df2["EFFECT"])
effect_pivot2 = pd.crosstab(dg2["ID"], dg2["EFFECT"])

# Convertir les comptes en booléens (0 ou 1)
effect_pivot_bool = (effect_pivot > 0).astype(int)
effect_pivot_bool2 = (effect_pivot2 > 0).astype(int)

# Filtrer pour garder que ces effets
effects_to_keep = ["inframe_codon_loss", "non_synonymous_codon"]
effect_pivot_filtered = effect_pivot_bool[effects_to_keep]
effect_pivot_filtered2 = effect_pivot_bool2[effects_to_keep]

# Fusionner avec df1
df1 = df1.merge(effect_pivot_filtered, left_on="ID", right_index=True, how="left")
dg1 = dg1.merge(effect_pivot_filtered2, left_on="ID", right_index=True, how="left")
# Remplacer les NaN par 0 (patients sans ces effets)
df1.fillna(0, inplace=True)
dg1.fillna(0, inplace=True)
df1, dg1

(           ID CENTER  BM_BLAST     WBC   ANC  MONOCYTES    HB    PLT  \
 0     P132697    MSK      14.0    2.80  0.20       0.70   7.6  119.0   
 1     P132698    MSK       1.0    7.40  2.40       0.10  11.6   42.0   
 2     P116889    MSK      15.0    3.70  2.10       0.10  14.2   81.0   
 3     P132699    MSK       1.0    3.90  1.90       0.10   8.9   77.0   
 4     P132700    MSK       6.0  128.00  9.70       0.90  11.1  195.0   
 ...       ...    ...       ...     ...   ...        ...   ...    ...   
 3318  P121828     VU       1.0    3.70  2.53       0.53   8.9  499.0   
 3319  P121829     VU       0.0    4.20  2.40       0.22  10.6   49.0   
 3320  P121830     VU       0.0    1.80  0.55       0.29   9.4   86.0   
 3321  P121853     VU       5.0    1.37  0.37       0.11  11.4  102.0   
 3322  P121834     VU       0.0    2.70  0.72       0.23   8.2  239.0   
 
                                            CYTOGENETICS  MUTATION  VAF_TOT  \
 0                       46,xy,del(20)(q12)

In [113]:
effects_cols = df1.columns[11:]

# Convertir ces colonnes en booléens
df1[effects_cols] = df1[effects_cols].astype(bool)
dg1[effects_cols] = dg1[effects_cols].astype(bool)

In [114]:
from sksurv.util import Surv

# Drop rows where 'OS_YEARS' is NaN if conversion caused any issues
target_df.dropna(subset=['OS_YEARS', 'OS_STATUS'], inplace=True)

# Check the data types to ensure 'OS_STATUS' is boolean and 'OS_YEARS' is numeric
print(target_df[['OS_STATUS', 'OS_YEARS']].dtypes)

# Contarget_dfvert 'OS_YEARS' to numeric if it isn’t already
target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')

# Ensure 'OS_STATUS' is boolean
target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

# 🔁 Filtrer df1 pour ne garder que les ID présents dans le target nettoyé
df1_filtered = df1[df1['ID'].isin(target_df['ID'])].copy()

# 🔁 Réindexer target_df pour qu'il soit dans le même ordre que df1_filtered
target_filtered = target_df.set_index("ID").loc[df1_filtered["ID"]].reset_index()

y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_filtered)

OS_STATUS    float64
OS_YEARS     float64
dtype: object


In [115]:
from sklearn.preprocessing import StandardScaler

# 1. On enlève les colonnes non souhaitées
X = df1_filtered .drop(columns=["ID", "CENTER", "CYTOGENETICS"])
X2 = dg1.drop(columns=["ID", "CENTER", "CYTOGENETICS"])

# 2. On identifie les colonnes numériques (float ou int)
numeric_cols = X.select_dtypes(include=["float", "int"]).columns

# 3. Standardisation des colonnes numériques
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
X2[numeric_cols] = scaler.fit_transform(X2[numeric_cols])

X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3173 entries, 0 to 3322
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   BM_BLAST              3173 non-null   float64
 1   WBC                   3173 non-null   float64
 2   ANC                   3173 non-null   float64
 3   MONOCYTES             3173 non-null   float64
 4   HB                    3173 non-null   float64
 5   PLT                   3173 non-null   float64
 6   MUTATION              3173 non-null   float64
 7   VAF_TOT               3173 non-null   float64
 8   SEX_MALE              3173 non-null   bool   
 9   inframe_codon_loss    3173 non-null   bool   
 10  non_synonymous_codon  3173 non-null   bool   
dtypes: bool(3), float64(8)
memory usage: 232.4 KB


In [116]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [117]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored , concordance_index_ipcw

# Initialize and train the Cox Proportional Hazards model
cox = CoxPHSurvivalAnalysis()
cox.fit(X_train, y_train)

# Evaluate the model using Concordance Index IPCW
cox_cindex_train = concordance_index_ipcw(y_train, y_train, cox.predict(X_train), tau=7)[0]
cox_cindex_test = concordance_index_ipcw(y_train, y_test, cox.predict(X_test), tau=7)[0]
print(f"Cox Proportional Hazard Model Concordance Index IPCW on train: {cox_cindex_train:.2f}")
print(f"Cox Proportional Hazard Model Concordance Index IPCW on test: {cox_cindex_test:.2f}")

Cox Proportional Hazard Model Concordance Index IPCW on train: 0.67
Cox Proportional Hazard Model Concordance Index IPCW on test: 0.67


In [123]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_ipcw

# Initialiser et entraîner le modèle Random Survival Forest
rsf = RandomSurvivalForest(
    n_estimators=1000,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)
rsf.fit(X_train, y_train)

# Prédictions (fonction de survie estimée -> on prend le risque à un temps donné, ici tau)
pred_train = rsf.predict(X_train)
pred_test = rsf.predict(X_test)

# Évaluation avec l'indice de concordance IPCW
rsf_cindex_train = concordance_index_ipcw(y_train, y_train, pred_train, tau=7)[0]
rsf_cindex_test = concordance_index_ipcw(y_train, y_test, pred_test, tau=7)[0]

print(f"Random Survival Forest Concordance Index IPCW on train: {rsf_cindex_train:.2f}")
print(f"Random Survival Forest Concordance Index IPCW on test: {rsf_cindex_test:.2f}")


Random Survival Forest Concordance Index IPCW on train: 0.77
Random Survival Forest Concordance Index IPCW on test: 0.69


In [122]:

rsf1 = RandomSurvivalForest(
    n_estimators=1000,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rsf1.fit(X, y)

pred = rsf1.predict(X2)
submission = pd.Series(pred, index=dg1['ID'], name='risk_score')
submission.to_csv('./first_rsf3.csv')

In [120]:
from sklearn.feature_selection import mutual_info_regression
import pandas as pd

# Extraire y en tant que vecteur numérique depuis l'objet Surv
# (y est probablement un StructuredArray avec les champs "event" et "time")
y_os_years = y["OS_YEARS"] if "OS_YEARS" in y.dtype.names else y["time"]

# Créer un masque pour ne garder que les lignes sans NaN
mask = ~np.isnan(y_os_years)

# Nettoyer X et y
X_clean = X.loc[mask]
y_clean = y_os_years[mask]

# Calculer l'information mutuelle
mi = mutual_info_regression(X_clean, y_clean, discrete_features='auto', n_neighbors=3, random_state=42)

# Résumé
mi_scores = pd.Series(mi, index=X_clean.columns).sort_values(ascending=False)
print(mi_scores)


PLT                     0.052893
MONOCYTES               0.034664
BM_BLAST                0.027325
HB                      0.016542
WBC                     0.013969
ANC                     0.013622
VAF_TOT                 0.012979
MUTATION                0.012215
SEX_MALE                0.011290
inframe_codon_loss      0.006876
non_synonymous_codon    0.004780
dtype: float64
